### An exploration of clustering methods to derive insights from a heart rate dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering

##### Data loading

##### Data preprocessing

##### Data exploration 

##### K-means clustering

##### Agglomerative clustering

##### Results